# Overview optimal remote restoring via controlled interaction with environment

In [1]:
from IPython.display import display

In [2]:
import numpy as np
import sympy as sp
import timeti

from quanty.state.coherence import coherence_matrix
from quanty.geometry import UniformChain
from quanty.hamiltonian import XX, XXZ
from quanty.model.homo import Homogeneous
from quanty.state.coherence import coherence_matrix
from quanty.task.transfer_ import FitTransmissionTimeTask

In [3]:
import sys, pathlib

sys.path.append(str(pathlib.Path.cwd().parent))

In [4]:
%load_ext autoreload
%autoreload 2

from orrvciwe import TransferProblem, TransferTask

## Overvew transfer

**Create a problem**

In [5]:
length = 5
n_sender = 2
excitations = 1

In [6]:
geometry = UniformChain()
model = Homogeneous(geometry)  # All with all !
hamiltonian = XXZ(model)
problem = TransferProblem.init_classic(
    hamiltonian,
    length=length,
    n_sender=n_sender,
    excitations=excitations,
)

**Check XXZ Hamiltonian**

In [7]:
sp.Matrix(problem.hamiltonian(length, excitations)).evalf(3)

Matrix([
[-2.23,       0,       0,      0,       0,       0],
[    0,   -1.05,     0.5, 0.0625,  0.0185, 0.00781],
[    0,     0.5, -0.0703,    0.5,  0.0625,  0.0185],
[    0,  0.0625,     0.5, 0.0177,     0.5,  0.0625],
[    0,  0.0185,  0.0625,    0.5, -0.0703,     0.5],
[    0, 0.00781,  0.0185, 0.0625,     0.5,   -1.05]])

**Fit transmission time**

In [8]:
task = FitTransmissionTimeTask(problem)
with timeti.profiler():
    result = task.run()
print(result.transmission_time)

Elapsed time of block: 14 ms
4.56617


**Fix tunning time**

In [9]:
tuning_time = 3
transmission_time = result.transmission_time - tuning_time

**Check unitary transform of main evolution by XXZ Hamiltonian**

In [10]:
sp.Matrix(problem.U(transmission_time)).evalf(3)

Matrix([
[-0.938 - 0.347*I,                    0,                  0,                 0,                  0,                    0],
[               0,     -0.196 + 0.766*I,    0.437 - 0.353*I,   -0.163 - 0.17*I, -0.0648 + 0.0169*I, -0.00704 + 0.00692*I],
[               0,      0.437 - 0.353*I,   0.512 - 0.0213*I, -0.0539 - 0.581*I,  -0.269 - 0.0663*I,   -0.0648 + 0.0169*I],
[               0,      -0.163 - 0.17*I,  -0.0539 - 0.581*I,  0.455 + 0.0169*I,  -0.0539 - 0.581*I,      -0.163 - 0.17*I],
[               0,   -0.0648 + 0.0169*I,  -0.269 - 0.0663*I, -0.0539 - 0.581*I,   0.512 - 0.0213*I,      0.437 - 0.353*I],
[               0, -0.00704 + 0.00692*I, -0.0648 + 0.0169*I,   -0.163 - 0.17*I,    0.437 - 0.353*I,     -0.196 + 0.766*I]])

**Create transfer task**

In [11]:
features = (
    (0, 0, 0.5, 1.5, 1.9),
    (0, 0, 1.4, 0.4, 2.2),
    (0, 0, 0.2, 1.2, 0.7),
)

task = TransferTask(
    problem=problem,
    transmission_time=transmission_time,
    tuning_time=tuning_time,
    features=features,
)

**Review first order coherence matrix**

In [12]:
sp.Matrix(coherence_matrix(order=1, basis=problem.sender_basis)[-1]).evalf(3)

Matrix([
[          0, x01 + I*y01, x02 + I*y02],
[x01 - I*y01,           0,           0],
[x02 - I*y02,           0,           0]])

**Check problem sender state**

In [13]:
print(task.problem.sender_params)

{r01: (0, 1), r02: (0, 2)}


In [14]:
sp.Matrix(task.problem.sender_state)

Matrix([
[             0, r02, r01],
[conjugate(r02),   0,   0],
[conjugate(r01),   0,   0]])

**Initial state of problem**

In [15]:
sp.Matrix(task.problem.initial_state)

Matrix([
[             0, 0, 0, 0, r02, r01],
[             0, 0, 0, 0,   0,   0],
[             0, 0, 0, 0,   0,   0],
[             0, 0, 0, 0,   0,   0],
[conjugate(r02), 0, 0, 0,   0,   0],
[conjugate(r01), 0, 0, 0,   0,   0]])

**Overview receiver state impacts**

In [16]:
with timeti.profiler():
    impacts = task.receiver_state_impacts(use_cache=False).items()

Elapsed time of block: 6 ms


In [17]:
[(v1, impact1), (v2, impact2)] = list(impacts)

display(v1)
display(sp.Matrix(impact1).evalf(6))
assert task._residuals(v1, impact1)[0] == impact1[0, 2].real
assert task._residuals(v1, impact1)[1] == impact1[0, 2].imag
display(task._residuals(v1, impact1))

display(v2)
display(sp.Matrix(impact2).evalf(6))
assert task._residuals(v2, impact2)[0] == impact2[0, 1].real
assert task._residuals(v2, impact2)[1] == impact2[0, 1].imag
display(task._residuals(v2, impact2))

task.perfect_transferred_state_residuals()

r01

Matrix([
[                      0, 0.0102535 + 0.0917825*I, 0.0979117 + 0.10008*I],
[0.0102535 - 0.0917825*I,                       0,                     0],
[  0.0979117 - 0.10008*I,                       0,                     0]])

(np.float64(0.09791167429957363), np.float64(0.10007965361946632))

r02

Matrix([
[                     0, 0.0255478 + 0.155741*I, 0.0877139 + 0.246526*I],
[0.0255478 - 0.155741*I,                      0,                      0],
[0.0877139 - 0.246526*I,                      0,                      0]])

(np.float64(0.025547839530431604), np.float64(0.1557406821883558))

array([0.09791167, 0.10007965, 0.02554784, 0.15574068])

In [18]:
print(*sum(v * i for v, i in impacts)[0, 1:], sep="\n")

r01*(0.0102535037251317 + 0.0917824441431267*I) + r02*(0.0255478395304316 + 0.155740682188356*I)
r01*(0.0979116742995736 + 0.100079653619466*I) + r02*(0.0877138595782704 + 0.246525748551345*I)


**Overview receiver state**

In [19]:
sp.Matrix(task.receiver_state()).evalf(3)

Matrix([
[                                               0, r01*(0.0103 + 0.0918*I) + r02*(0.0255 + 0.156*I), r01*(0.0979 + 0.1*I) + r02*(0.0877 + 0.247*I)],
[r01*(0.0103 - 0.0918*I) + r02*(0.0255 - 0.156*I),                                                0,                                             0],
[   r01*(0.0979 - 0.1*I) + r02*(0.0877 - 0.247*I),                                                0,                                             0]])

**Overvew residuals**

In [20]:
print(
    task.perfect_transferred_state_residuals(), np.sum(task.perfect_transferred_state_residuals())
)

[0.09791167 0.10007965 0.02554784 0.15574068] 0.37927984963782735


## Overvew optimization

In [21]:
from scipy import optimize
from quanty.optimize import brute_random

In [22]:
n_interacted_with_enviroment = 3
x0 = np.array([0.1, 0.2, 0.3])
x_bounds = ((0, 1), (0, 1), (0, 1))


def fun(x):
    _features = x.reshape(-1, n_interacted_with_enviroment)
    n_peaks = _features.shape[0]
    if _features.shape[1] != length:
        _features = np.hstack(
            (np.zeros((n_peaks, length - n_interacted_with_enviroment)), _features)
        )
    _features = tuple([tuple(row) for row in _features])

    _task = TransferTask(
        problem=problem,
        transmission_time=transmission_time,
        tuning_time=tuning_time,
        features=_features,
    )
    r = np.sum(_task.perfect_transferred_state_residuals())
    return r

**Minimize with L-BFGS-B**

In [23]:
res = optimize.minimize(fun, x0, bounds=x_bounds, method="L-BFGS-B")
res

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.3729342354765576
        x: [ 1.000e+00  1.000e+00  1.000e+00]
      nit: 3
      jac: [-1.463e-01 -1.624e-01 -2.445e-02]
     nfev: 24
     njev: 6
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>

**Search optimal with bruteforce**

In [24]:
%%time
res = brute_random(fun, ranges=x_bounds, verbose=False, no_local_search=True, maxiter=int(1e4))
res

CPU times: user 26.7 s, sys: 257 ms, total: 27 s
Wall time: 26.9 s


 fun: 0.37846435748740803
   x: [ 9.993e-01  9.880e-01  8.574e-01]